### TODO:
1. For each month, sort articles by viewcount
2. Limit to top 1000 articles

In [1]:
# import libraries
import pandas as pd
import os
from tempfile import mkstemp
from shutil import move
from os import fdopen, remove
import pickle
import numpy as np
import sqlalchemy as sql
import csv
from tqdm import tqdm_notebook
import wikipedia
from multiprocessing import Pool
import datetime


In [2]:
DIR = "/home/justina/Desktop/dv/data_viz/server/wiki-data/2016-2018"
lst_data= os.listdir(DIR)


In [ ]:
def replace(file_path):

    fh, abs_path = mkstemp()
    with fdopen(fh, 'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                head = line.split(';')[0:-2]
                tail= line.split(';')[-2:]
                tail = ' '.join(tail)
                head = ';'.join(head)
                final = head + ' ' + tail
                new_file.write(final)
    old = len(open(file_path).readlines())
    new = len(open(abs_path).readlines())
    print("Number of lines: ")
    print("{}, Old: {}, New: {}".format(file_path,old, new))
    
    if old == new:
        move(abs_path, file_path)
        
for file in lst_data:
    if 'pagecounts' in file:
        if 'spaces' not in file:
            replace(os.path.join(DIR, file))

In [3]:
def create_df(folder_of_files):
    # df of article name, monthly viewcount, day
    
    lst_data = os.listdir(folder_of_files)
    # lst_data=['pagecounts-2016-10-views-ge-5_cleaned']
    columns = ['article_name', 
               'monthly_viewcount', 
               'day']
    df = pd.DataFrame([])
    year_month = []
    for file in lst_data:
        if 'pagecounts' in file:
            next_df = pd.read_csv(os.path.join(folder_of_files, file), sep=' ',dtype={columns[0]:str, columns[1]:int, columns[2]:int}, header=None, names=columns)
            lst_filename = file.split('-')
            yr = lst_filename[1]
            mth = lst_filename[2]
            next_df['year'] = yr
            next_df['month'] = mth
            year_month.append((int(yr), int(mth)))
            next_df.sort_values(by='monthly_viewcount', ascending=False, inplace=True)
            print("Before filtering {}-{}: {}".format(yr,mth,next_df.year.size))
            mask = next_df.article_name.str.contains('Special:') | \
                    next_df.article_name.str.contains('Main_Page') | \
                    next_df.article_name.str.contains('Portal:') | \
                    next_df.article_name.str.contains('Wikipedia:') |\
                    next_df.article_name.str.contains('List of ') |\
                    next_df.article_name.str.contains('Special%:') |\
                    next_df.article_name.str.contains('User:') |\
                    next_df.article_name.str.contains('Help:') |\
                    next_df.article_name.str.contains('Category:') |\
                    next_df.article_name.str.contains('-') |\
                    next_df.article_name.str.contains('404.php') |\
                    next_df.article_name.str.contains('File:')
            
            next_df=next_df[~mask]
            print("After filtering: {}".format(next_df.year.size))
            df = pd.concat([df, next_df.head(1000)], ignore_index = True)
    
    df = df.astype({'article_name': 'str', 'monthly_viewcount': 'int64', 'day': 'int64', 'year': 'int64', 'month':'int64'})
    df['peak_date'] = pd.to_datetime(df[['year', 'month', 'day']])
    return df



In [4]:

# df = df.astype({'article_name': 'str', 'monthly_viewcount': 'int64', 'day': 'int64', 'year': 'int64', 'month':'int64'})
# df['peak_date'] = pd.to_datetime(df[['year', 'month', 'day']])
# df.head()

df = create_df(os.path.join(DIR, 'final'))

Before filtering 2016-06: 51023
After filtering: 48840
Before filtering 2016-11: 54346
After filtering: 51972
Before filtering 2017-06: 44463
After filtering: 42485
Before filtering 2017-09: 44067
After filtering: 42162
Before filtering 2016-01: 60366
After filtering: 57826
Before filtering 2017-11: 48770
After filtering: 46619
Before filtering 2016-04: 56701
After filtering: 54211
Before filtering 2017-12: 43520
After filtering: 41585
Before filtering 2017-01: 56928
After filtering: 54463
Before filtering 2018-05: 45374
After filtering: 43333
Before filtering 2016-02: 57164
After filtering: 54676
Before filtering 2018-06: 38406
After filtering: 36742
Before filtering 2017-04: 49386
After filtering: 47091
Before filtering 2018-07: 39684
After filtering: 37916
Before filtering 2016-08: 50317
After filtering: 48152
Before filtering 2018-11: 40655
After filtering: 38877
Before filtering 2018-01: 50493
After filtering: 48273
Before filtering 2016-12: 50913
After filtering: 48643
Before fil

In [5]:
df.groupby(['year', 'month']).count()

article_name  monthly_viewcount   day  peak_date
year month                                                  
2016 1              1000               1000  1000       1000
     2              1000               1000  1000       1000
     3              1000               1000  1000       1000
     4              1000               1000  1000       1000
     5              1000               1000  1000       1000
     6              1000               1000  1000       1000
     7              1000               1000  1000       1000
     8              1000               1000  1000       1000
     9              1000               1000  1000       1000
     10             1000               1000  1000       1000
     11             1000               1000  1000       1000
     12             1000               1000  1000       1000
2017 1              1000               1000  1000       1000
     2              1000               1000  1000       1000
     3              1000               1000  1000       1000
     4              1000               1000  1000       1000
     5              1000               1000  1000       1000
     6              1000               1000  1000       1000
     7              1000               1000  1000       1000
     8              1000               1000  1000       1000
     9              1000               1000  1000       1000
     10             1000               1000  1000       1000
     11             1000               1000  1000       1000
     12             1000               1000  1000       1000
2018 1              1000               1000  1000       1000
     2              1000               1000  1000       1000
     3              1000               1000  1000       1000
     4              1000               1000  1000       1000
     5              1000               1000  1000       1000
     6              1000               1000  1000       1000
     7              1000               1000  1000       1000
     8              1000               1000  1000       1000
     9              1000               1000  1000       1000
     10             1000               1000  1000       1000
     11             1000               1000  1000       1000

In [6]:
wikipedia.set_lang('en')
wikipedia.set_rate_limiting(rate_limit = True)
def get_article_data(x):
    row = x
    try:
        name = row['article_name']
        page = wikipedia.WikipediaPage(title=name)
        return name, page, row
        
    except:
        name = row['article_name']
        print(name)
        return name, None, row

def insert(lst, con, articles, article_name, year, month, day, view_count, peak_date, summary, page_id):
    lst.append({'title': article_name,
                   'year': year,
                   'month':month,
                   'day':day,
                   'view_count':view_count,
                   'peak_date': peak_date,
                   'summary': summary,
                   'page_id': page_id})
    if len(lst) > 50:
        print("Inserting...")
        con.execute(articles.insert(), lst)
#         except:
#             print("Did not insert")
        lst = []
    return lst
    
    
    

                    

In [39]:
fromdb_df = pd.read_csv('./2016-2018/top_1000_2016-2018/fromdb_data.csv', names = ['article_name', 'summary', 'page_id'])
fromdb_df.head()

,article_name,summary,page_id
0,-,The hyphen-minus (-) is a character used in di...,2734201
1,404.php,"The HTTP 404, 404 Not Found, and 404 error mes...",35507
2,AMGTV,AMGTV is an American family-oriented televisio...,18746613
3,Proyecto_40,XHTVM-TDT (virtual channel 40) is a television...,6948474
4,Donald_Trump,"Donald John Trump (born June 14, 1946) is the ...",4848272


In [45]:
'404.php' in fromdb_df.article_name
fromdb_df.loc[1].article_name

article_names = set(fromdb_df.article_name.values)

In [65]:
'404.php' in article_names
# summary = fromdb_df[fromdb_df['article_name']=='adsf'].summary.values[0]
print(fromdb_df[fromdb_df['article_name']=='404.php'].summary.values[0])

The HTTP 404, 404 Not Found, and 404 error message is a Hypertext Transfer Protocol (HTTP) standard response code, in computer network communications, to indicate that the client was able to communicate with a given server, but the server could not find what was requested.
The website hosting server will typically generate a "404 Not Found" web page when a user attempts to follow a broken or dead link; hence the 404 error is one of the most recognizable errors encountered on the World Wide Web.


In [53]:
def get_final_data(filepath):    
    # df has title, year, month, day, viewcount, peak date
    fromdb_df = pd.read_csv('./2016-2018/top_1000_2016-2018/fromdb_data.csv', names = ['article_name', 'summary', 'page_id'], na_values = '')
    
    article_names = set(fromdb_df.article_name.values)
    new_df = pd.DataFrame(columns = ['title', 'year', 'month', 'day', 'view_count', 'peak_date', 'summary', 'page_id'])
    df = pd.read_csv(filepath)
    exists = []
    for _, row in tqdm_notebook(df.iterrows(), total = len(df)):
        try:
            title = row['article_name']
            year = row['year']
            month = row['month']
            day = row['day']
            view_count =row['monthly_viewcount']
            peak_date = row['peak_date']

            if (title in article_names):

                summary = fromdb_df[fromdb_df['article_name']==title].summary.values[0]
                page_id = fromdb_df[fromdb_df['article_name']==title].page_id.values[0]
            else:
                article_names.add(title)
                _, page, _ = get_article_data(row)
                if page:
                    summary = page.summary
                    page_id = page.pageid
                    fromdb_df.append({'article_name': title,
                                     'summary': summary,
                                     'page_id': page_id}, ignore_index = True)
                else:
                    summary = ""
                    page_id = -1
                    fromdb_df.append({'article_name': title,
                                     'summary':summary,
                                     'page_id':page_id}, ignore_index = True)
            new_df = new_df.append({'title': title,
                          'year': year,
                          'month': month,
                          'day': day,
                          'view_count': view_count,
                          'peak_date': peak_date,
                          'summary': summary,
                          'page_id': page_id}, ignore_index= True)
        e

    new_df.to_csv('./2016-2018/top_1000_2016-2018/final_data_0.csv')
    return new_df

In [54]:
get_final_data('./2016-2018/top_1000_2016-2018/partial_view_data0.csv')

KeyboardInterrupt: 

In [33]:
len(set(df.article_name.unique()) - set(fromdb_df.article_name.unique()))

2339

In [34]:
df.to_csv('./wiki-data/2016-2018/top_1000_2016-2018/view_data.csv')
df_array = np.array_split(df,2)
for i, dframe in enumerate(df_array):
    dframe.to_csv('./wiki-data/2016-2018/top_1000_2016-2018/partial_view_data{}.csv'.format(i))

In [16]:
x[:2]

[1, 2]

## Read the data from top 1000 per month
TOP1000_DIR = "/home/justina/Desktop/dv/data_viz/server/wiki-data/top_1000_2016-2018"

df1 = pd.DataFrame([])
for f in os.listdir(TOP1000_DIR):
    if 'counts' in f:
        df1 = pd.concat([df1, pd.read_csv(os.path.join(TOP1000_DIR, f))])

In [ ]:
df1.year.size

In [ ]:
def print_rows(limit=10):
    i = 0
    with engine.connect() as con:
        rows = con.execute(articles.select())
        for r in rows:
            print(r.title, int(r.view_count))
            i += 1
            if ( i == limit): return

In [ ]:
print_rows()

In [ ]:
urls_folder_path = "/home/justina/Desktop/dv/data_viz/server/wiki/"

links = open(os.path.join(urls_folder_path, 'urls_all.txt'),'r').readlines()
for i in range(0, len(links),5):
    fh, abs_path = mkstemp()
    with fdopen(fh, 'w') as new_file:
        print(i, i+5)
        for j in links[i:i+5]:
            new_file.write(j)
    move(abs_path, os.path.join(urls_folder_path, 'url{}.txt'.format(i)))

        
    

## Add may 2017 data to db again

In [ ]:
TOP1000_DIR = "/home/justina/Desktop/dv/data_viz/server/wiki-data/top_1000_per_month"

df1 = pd.DataFrame([])
for f in os.listdir(TOP1000_DIR):
    if 'counts' in f:
        df1 = pd.concat([df1, pd.read_csv(os.path.join(TOP1000_DIR, f))], ignore_index = True)
        
df1.year.size

In [ ]:
df1[(df1.month == 5) & (df1.year == 2017)].head(50)[['article_name', 'peak_date']]

In [ ]:
def add_entries(engine_path, df):
    engine = sql.create_engine(engine_path)
    metadata = sql.MetaData()
    articles = sql.Table('articles', metadata,
        sql.Column('title', sql.String),
        sql.Column('year', sql.Integer),
        sql.Column('month', sql.Integer),
        sql.Column('day', sql.Integer),
        sql.Column('view_count', sql.Integer), 
        sql.Column('peak_date', sql.Date),
        sql.Column('summary', sql.Text),
        sql.Column('page_id', sql.Integer)
    )
    metadata.create_all(engine)

    result = []
    errors = []
    wiki_df = pd.DataFrame(columns=['article_name', 'summary', 'page_id'])

    with engine.connect() as con:
        lst = list(df.iterrows())
    #     with Pool(50) as pool:
    #     for each in tqdm_notebook(pool.imap(get_article_data, lst), total=len(lst)):
        for _, row in lst:
            name = row['article_name']
            year = row['year']
            month = row['month']
            day = row['day']
            view_count = row['monthly_viewcount']
            peak_date = datetime.datetime.strptime(row['peak_date'], '%Y-%m-%d').date()

            if name not in wiki_df.article_name.values:
                name, page, r = get_article_data(row)

                if page:
                    wiki_df = wiki_df.append({'article_name': name, 'summary': page.summary, 'page_id': page.pageid}, ignore_index = True)
                else:
                    wiki_df = wiki_df.append({'article_name': name, 'summary': "", 'page_id': -1}, ignore_index = True)

                result = insert(result,con, articles, name, year, month, day, view_count, peak_date, wiki_df[wiki_df['article_name'] == name].summary.values[0], wiki_df[wiki_df['article_name'] == name].page_id.values[0])
            

        con.execute(articles.insert(), result)
                
        return wiki_df

                

In [ ]:
wiki_df=add_entries('sqlite:///test.db', df1.loc[100:150])

In [ ]:
# Create dataframe of all the data

In [ ]:
wiki_df

In [ ]:
engine = sql.create_engine('sqlite:///test.db')
metadata = sql.MetaData()
articles = sql.Table('articles', metadata,
    sql.Column('title', sql.String),
    sql.Column('year', sql.Integer),
    sql.Column('month', sql.Integer),
    sql.Column('day', sql.Integer),
    sql.Column('view_count', sql.Integer), 
    sql.Column('peak_date', sql.Date),
    sql.Column('summary', sql.Text),
    sql.Column('page_id', sql.Integer)
)
metadata.create_all(engine)

result = []
errors = []
wiki_df = pd.DataFrame(columns=['article_name', 'summary', 'page_id'])

with engine.connect() as con:
    name = df1.loc[0,'article_name']
    year = df1.loc[0,'year']
    month = df1.loc[0,'month']
    day = df1.loc[0,'day']
    view_count = df1.loc[0,'monthly_viewcount']
    peak_date = datetime.datetime.strptime(df1.loc[0,'peak_date'], '%Y-%m-%d').date()

    name, page, row = get_article_data((df1.loc[0]))
    con.execute(articles.insert(), {'title': name,
                                   'year': year,
                                   'month': month,
                                   'day': day,
                                   'view_count': view_count,
                                   'peak_date': peak_date,
                                   'summary': "",
                                   'page_id': -1})

In [ ]:
df1.loc[0,'peak_date']

In [ ]:
wiki_df = add_entries('sqlite:///database_working_add_052017.db',df1)

In [ ]:
df1.dtypes

In [ ]:
wiki_df = add_entries('sqlite:///database_working_add_052017.db',df1[851:])

In [ ]:
df1[df1.article_name == 'Restoration_(Spain)']